In [14]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import warnings 
from tensorflow.examples.tutorials.mnist import input_data


warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')

In [15]:
location = 'E:/Dataset/MNIST/'
mnist = input_data.read_data_sets(train_dir=location,one_hot=True)
print('ok!')

Extracting E:/Dataset/MNIST/train-images-idx3-ubyte.gz
Extracting E:/Dataset/MNIST/train-labels-idx1-ubyte.gz
Extracting E:/Dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting E:/Dataset/MNIST/t10k-labels-idx1-ubyte.gz
ok!


In [16]:
#超参数
learning_rate = 0.01
num_step = 500
batch_size = 128
display_step = 20


#网络参数
input_size = 784 # 28 X 28 图片
num_class = 10  # MNIST类别为 0-9 共10类
drop_out =0.75 #随机关断75%的神经元


#tensorflow 图结构的输入
X = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,num_class])
keep_prob = tf.placeholder(tf.float32)

In [17]:
#定义一个卷积函数
def conv2d(x,W,b,strides =1):
    conv_result = tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    add_bias = tf.nn.bias_add(conv_result,b)
    return tf.nn.relu(add_bias)

def maxpooling2d(x,k=2):
    #最大池化,k x k 池化核，步长为k
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

#搭建卷积神经网络
def convolution_net(x,weights,biases,dropout_parame):
    
    # 将784  reshape 为  【batch_size, height ,width, channel】
    x_reshaped = tf.reshape(x,shape=[-1,28,28,1])
    
#---------------第1个卷积池化--------------------------------------------  
    conv1 = conv2d(x_reshaped,weights['wc1'],biases['bc1'])#卷积层
    pooling1 = maxpooling2d(conv1,k=2) #池化层
#-----------------------------------------------------------  
    

#---------------第2个卷积池化--------------------------------------------  
    conv2 = conv2d(pooling1,weights['wc2'],biases['bc2'])#卷积层
    pooling2 = maxpooling2d(conv2,k=2) #池化层
#-----------------------------------------------------------  
    
#---------------全连接层---------------------------
#     Reshape conv2 output to fit fully connected layer input，调整conv2层输出的结果以符合全连接层的需求
    fc = tf.reshape(pooling2,[-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc,weights['wd1']),biases['bd1'])
    fc1_2 = tf.nn.relu(fc1)
    fc_layer = tf.nn.dropout(fc1_2,dropout_parame) #dropout
    
    #输出层
    out = tf.add(tf.matmul(fc_layer,weights['out']),biases['out'])
    return out

In [18]:
#网络参数
weights = {
     # 5x5 conv, 1 input, 32 outputs
    'wc1':tf.Variable(tf.random_normal([5,5,1,32]),dtype=tf.float32),
     # 5x5 conv, 32 inputs, 64 outputs
    'wc2':tf.Variable(tf.random_normal([5,5,32,64]),dtype=tf.float32),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1':tf.Variable(tf.random_normal([7*7*64,1024]),dtype=tf.float32),
     # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_class]),dtype=tf.float32)
}
biases = {
     #对第一层每个卷积神经元的输出加上偏执，32个
    'bc1':tf.Variable(tf.random_normal([32]),dtype=tf.float32),
     #对第一层每个卷积神经元的输出加上偏执，64个
    'bc2':tf.Variable(tf.random_normal([64]),dtype=tf.float32),
     #对全连接层加上偏执，1024个
    'bd1':tf.Variable(tf.random_normal([1024]),dtype=tf.float32),
     #1024输入，num_class 个输出
    'out':tf.Variable(tf.random_normal([num_class]),dtype=tf.float32)
}

#构建模型
logits  = convolution_net(X,weights,biases,keep_prob)
prediction = tf.nn.softmax(logits)

#定义损失函数和优化函数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(loss_op)

#定义进行模型评估的方法
correctct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correctct_prediction,tf.float32))

#初始化
init = tf.global_variables_initializer()


In [19]:
with tf.Session() as sess:
    sess.run(init)
    for steps in range(num_step):
        batch = mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict = {X:batch[0], y:batch[1],keep_prob:drop_out})
        
        if (steps % display_step) ==0:
            loss,acc = sess.run([loss_op,accuracy],feed_dict = { X:batch[0],y:batch[1],keep_prob:drop_out})
            print('step '+str(steps)+',Minihatch loss: '+ '{:.4f}'.format(loss) + \
                 '  Training Accuracy :'+ '{:.4f}'.format(acc))
    print('\nOptimization Finished !\n')
    print('Test Accuracy : '+ str(sess.run(accuracy,feed_dict={ X : mnist.test.images[:256],
                                                                y : mnist.test.labels[:256],
                                                                keep_prob:drop_out})))

step 0,Minihatch loss: 113789.7969  Training Accuracy :0.1328
step 20,Minihatch loss: 3473.7319  Training Accuracy :0.6719
step 40,Minihatch loss: 655.5287  Training Accuracy :0.8672
step 60,Minihatch loss: 487.5071  Training Accuracy :0.8750
step 80,Minihatch loss: 638.9553  Training Accuracy :0.8906
step 100,Minihatch loss: 585.3985  Training Accuracy :0.8359
step 120,Minihatch loss: 478.7036  Training Accuracy :0.8828
step 140,Minihatch loss: 174.8081  Training Accuracy :0.9062
step 160,Minihatch loss: 354.0918  Training Accuracy :0.8984
step 180,Minihatch loss: 211.2562  Training Accuracy :0.9453
step 200,Minihatch loss: 177.2878  Training Accuracy :0.9453
step 220,Minihatch loss: 246.1749  Training Accuracy :0.8984
step 240,Minihatch loss: 167.8315  Training Accuracy :0.9531
step 260,Minihatch loss: 112.7510  Training Accuracy :0.9609
step 280,Minihatch loss: 183.6189  Training Accuracy :0.9219
step 300,Minihatch loss: 103.7153  Training Accuracy :0.9531
step 320,Minihatch loss: 1